# Evaluation

In [277]:
import itertools # 是 Python 的內建模組，提供了一組用於處理迭代器的函數和工具。
                 # 它包含了各種用於高效處理迭代器的函數，可以幫助我們編寫更簡潔、高效的代碼。
import sys # 是 Python 的內建模組，提供了與 Python 解釋器和運行環境相關的功能。

import numpy as np

# sys.path 是一個列表，包含了 Python 解釋器在導入模組時會搜尋的路徑。
# 當你使用 import 語句導入模組時 Python 會依次在 sys.path 中的路徑下尋找對應的模組文件。
sys.path.append("../ecg-classification/")
# sys.path.append("C:\\Users\\Chen_Lab01\\Documents\\GitHub/ecg-classification")
# from IPython.display import Video
# import cv2
from matplotlib import pyplot as plt
plt.style.use("ggplot") #  是 Matplotlib 庫中用於設置繪圖樣式的函數。它使用了一種名為 "ggplot" 的預定義樣式
import matplotlib
matplotlib.use("TkAgg")
                        #  該樣式模仿了 R 語言的 ggplot2 繪圖包的外觀。
# print(sys.path)
import torch
from ecg_tools.config import EcgConfig, Mode
from ecg_tools.data_loader import DatasetConfig, get_data_loaders
from ecg_tools.model import ECGformer
from ecg_tools.train import ECGClassifierTrainer


## Load model

In [279]:
import torch
config = EcgConfig()    
model_quantized = torch.load("..\\..\\model_save\\model_quantized_98_torch.pth")
model = torch.load("..\\..\\model_save\\model_epoch_98.pth")
model_quantized.eval()
model_quantized.to('cpu')
model.eval()
model.to('cpu')

ECGformer(
  (encoder): ModuleList(
    (0-5): 6 x TransformerEncoderLayer(
      (0): ResidualAdd(
        (block): Sequential(
          (0): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
          (1): MultiHeadAttention(
            (queries_projection): Linear(in_features=192, out_features=192, bias=True)
            (values_projection): Linear(in_features=192, out_features=192, bias=True)
            (keys_projection): Linear(in_features=192, out_features=192, bias=True)
            (final_projection): Linear(in_features=192, out_features=192, bias=True)
          )
          (2): Dropout(p=0.1, inplace=False)
        )
      )
      (1): ResidualAdd(
        (block): Sequential(
          (0): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
          (1): MLP(
            (0): Linear(in_features=192, out_features=768, bias=True)
            (1): GELU(approximate='none')
            (2): Linear(in_features=768, out_features=192, bias=True)
          )
          (2):

## 量化模型

In [278]:
# import torch.quantization
# 
# # 使用 Eager Mode Quantization
# # 將 torch.nn.Linear 的參數映射到 -127~127 之間

# quantized_model = torch.quantization.quantize_dynamic(
#     model, {torch.nn.Linear}, dtype=torch.qint8
# )
# 
# torch.save(quantized_model, "..\\model_save\\model_quantized_98.pth")

## 準確度測試

In [281]:
import einops
loader = get_data_loaders(DatasetConfig())
accuracy = 0
for signal, label in loader[Mode.train]:
    signal.to('cpu')
    label.to('cpu')
    signal = einops.rearrange(signal, "b c e -> b e c")
    # print(signal)
    p = model_quantized(signal)
    print(p)
    print(label)
    print(signal.shape, label.shape)
    print(p.argmax(1) == label)
    accuracy += torch.sum(p.argmax(1) == label)
    print(f"accuracy: {accuracy / config.dataset.batch_size}")
    break



tensor([[ 5.7121e+00,  8.1317e-01, -2.1535e+00, -4.5101e+00, -2.4372e-01,
         -4.4972e+00],
        [ 5.5329e+00,  1.2125e+00, -6.6878e+00, -2.1944e+00, -1.3575e+01,
          5.9528e+00],
        [ 8.6056e+00,  1.4416e+00, -5.1934e+00, -3.1483e+00, -1.6442e+01,
          2.4495e+00],
        [ 5.2470e+00, -1.2009e+00, -4.3494e+00, -2.4378e+00, -3.9618e+00,
          1.3853e-01],
        [ 6.2940e+00, -1.9455e-01, -3.6662e+00, -4.9141e+00, -3.8721e+00,
         -9.5588e-02],
        [ 5.2444e+00, -5.3966e-01, -4.4465e+00, -3.9307e+00, -5.1170e+00,
          2.3582e+00],
        [ 6.3309e+00, -3.3984e-01, -4.0722e+00, -4.1203e+00, -5.1247e+00,
         -1.3886e-01],
        [ 3.6990e+00, -2.2362e+00, -1.7755e+00, -2.9430e+00, -3.9126e+00,
          1.6458e+00],
        [ 7.4732e+00, -1.4786e+00, -7.2334e+00, -1.5093e+00, -5.7579e+00,
          7.7000e-01],
        [ 4.2240e+00, -6.6191e-01, -2.4108e+00, -1.6537e+00,  5.4606e-01,
         -4.0633e+00],
        [ 6.7336e+00,  2.9788e

## View Prameter

Model layer

In [292]:
print(model_quantized)

ECGformer(
  (encoder): ModuleList(
    (0-5): 6 x TransformerEncoderLayer(
      (0): ResidualAdd(
        (block): Sequential(
          (0): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
          (1): MultiHeadAttention(
            (queries_projection): DynamicQuantizedLinear(in_features=192, out_features=192, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (values_projection): DynamicQuantizedLinear(in_features=192, out_features=192, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (keys_projection): DynamicQuantizedLinear(in_features=192, out_features=192, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (final_projection): DynamicQuantizedLinear(in_features=192, out_features=192, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
          )
          (2): Dropout(p=0.1, inplace=False)
        )
      )
      (1): ResidualAdd(
        (block): Sequential(
          (0): LayerNorm((192,), eps=1e-05, elementwise_affine=True)


字典形式的量化模型參數

In [283]:
# 量化
for param_name, param_tensor in model_quantized.state_dict().items():
    print(f"{param_name}")

positional_encoding
encoder.0.0.block.0.weight
encoder.0.0.block.0.bias
encoder.0.0.block.1.queries_projection.scale
encoder.0.0.block.1.queries_projection.zero_point
encoder.0.0.block.1.queries_projection._packed_params.dtype
encoder.0.0.block.1.queries_projection._packed_params._packed_params
encoder.0.0.block.1.values_projection.scale
encoder.0.0.block.1.values_projection.zero_point
encoder.0.0.block.1.values_projection._packed_params.dtype
encoder.0.0.block.1.values_projection._packed_params._packed_params
encoder.0.0.block.1.keys_projection.scale
encoder.0.0.block.1.keys_projection.zero_point
encoder.0.0.block.1.keys_projection._packed_params.dtype
encoder.0.0.block.1.keys_projection._packed_params._packed_params
encoder.0.0.block.1.final_projection.scale
encoder.0.0.block.1.final_projection.zero_point
encoder.0.0.block.1.final_projection._packed_params.dtype
encoder.0.0.block.1.final_projection._packed_params._packed_params
encoder.0.1.block.0.weight
encoder.0.1.block.0.bias
enco

字典形式的非量化模型參數

In [282]:
# 未量化
for param_name, param_tensor in model.state_dict().items():
    print(f"{param_name}")

positional_encoding
encoder.0.0.block.0.weight
encoder.0.0.block.0.bias
encoder.0.0.block.1.queries_projection.weight
encoder.0.0.block.1.queries_projection.bias
encoder.0.0.block.1.values_projection.weight
encoder.0.0.block.1.values_projection.bias
encoder.0.0.block.1.keys_projection.weight
encoder.0.0.block.1.keys_projection.bias
encoder.0.0.block.1.final_projection.weight
encoder.0.0.block.1.final_projection.bias
encoder.0.1.block.0.weight
encoder.0.1.block.0.bias
encoder.0.1.block.1.0.weight
encoder.0.1.block.1.0.bias
encoder.0.1.block.1.2.weight
encoder.0.1.block.1.2.bias
encoder.1.0.block.0.weight
encoder.1.0.block.0.bias
encoder.1.0.block.1.queries_projection.weight
encoder.1.0.block.1.queries_projection.bias
encoder.1.0.block.1.values_projection.weight
encoder.1.0.block.1.values_projection.bias
encoder.1.0.block.1.keys_projection.weight
encoder.1.0.block.1.keys_projection.bias
encoder.1.0.block.1.final_projection.weight
encoder.1.0.block.1.final_projection.bias
encoder.1.1.bloc

## Packed_params

In [284]:
import numpy as np
weight_tensor_after = model_quantized.state_dict()['encoder.0.0.block.1.queries_projection._packed_params._packed_params']

packed_params = model_quantized.encoder[0][0].block[1].queries_projection._packed_params._packed_params

# Unpack the quantized weights and biases
int8_weights, int8_bias = torch.ops.quantized.linear_unpack(packed_params)
int8_weights_nd = np.array(int8_weights.int_repr())
int8_bias_nd = int8_bias.detach().numpy()
# # Dequantize the weights and biases
# weights = int8_weights.dequantize()
# bias = int8_bias.dequantize()

print(int8_weights_nd)#　將量化後的權重轉換為整數表示並轉化為numpy# array
print(int8_weights.q_scale()) #　獲取量化的scale
print(int8_bias_nd)

[[-70  25  45 ...   9 -11  41]
 [ -1  -2 -10 ...  -6 -11 -24]
 [-69  75  15 ...  -9   9 -46]
 ...
 [ 42 -16  -7 ... -34 -45  27]
 [  3 -18  19 ...  11 -29  11]
 [ 33 -54 -41 ...  43 -23  35]]
0.0016688834875822067
[ 0.06134006  0.05202604 -0.03778487 -0.00852696 -0.04655216  0.09476199
 -0.01202581  0.02501333  0.00824877  0.00398409 -0.05725901 -0.02867818
  0.04962902 -0.0563751  -0.03398313  0.02636591 -0.04214004  0.02425917
 -0.03342133 -0.04976596  0.07020013  0.06184619  0.00831161  0.01872784
  0.04020224 -0.05760437 -0.00331612 -0.01521585 -0.02899045  0.07367223
 -0.00373852 -0.05436532 -0.07119824 -0.00342908  0.04767367  0.0072046
 -0.0268723   0.08635192 -0.059508   -0.04140051 -0.01219196  0.04398649
 -0.00661933  0.09846511 -0.06277379  0.05841304 -0.00993187  0.00212757
 -0.00176864 -0.01117148  0.01954278 -0.03350052 -0.06176341  0.0046997
 -0.0649123   0.00070314 -0.04784982 -0.00160643 -0.02818445 -0.05547581
  0.02812169 -0.03888802 -0.07212877  0.02438743  0.008075

In [285]:
for param_name, param_tensor in model_quantized.state_dict().items():
    if isinstance(param_tensor, torch.Tensor):
        if not any(special_param in param_name for special_param in ['scale', 'zero_point', 'dtype', '_packed_params']):
            print(f"{param_name}\t{param_tensor.size()}")

positional_encoding	torch.Size([188, 192])
encoder.0.0.block.0.weight	torch.Size([192])
encoder.0.0.block.0.bias	torch.Size([192])
encoder.0.1.block.0.weight	torch.Size([192])
encoder.0.1.block.0.bias	torch.Size([192])
encoder.1.0.block.0.weight	torch.Size([192])
encoder.1.0.block.0.bias	torch.Size([192])
encoder.1.1.block.0.weight	torch.Size([192])
encoder.1.1.block.0.bias	torch.Size([192])
encoder.2.0.block.0.weight	torch.Size([192])
encoder.2.0.block.0.bias	torch.Size([192])
encoder.2.1.block.0.weight	torch.Size([192])
encoder.2.1.block.0.bias	torch.Size([192])
encoder.3.0.block.0.weight	torch.Size([192])
encoder.3.0.block.0.bias	torch.Size([192])
encoder.3.1.block.0.weight	torch.Size([192])
encoder.3.1.block.0.bias	torch.Size([192])
encoder.4.0.block.0.weight	torch.Size([192])
encoder.4.0.block.0.bias	torch.Size([192])
encoder.4.1.block.0.weight	torch.Size([192])
encoder.4.1.block.0.bias	torch.Size([192])
encoder.5.0.block.0.weight	torch.Size([192])
encoder.5.0.block.0.bias	torch.S

印出模型中所有的weights和bias

In [286]:
for name, param in model_quantized.named_parameters():
    if 'weight' in name:
        print(f"Layer: {name}")
        print(f"Weight: {param.data}")
    elif 'bias' in name:
        print(f"Layer: {name}") 
        print(f"Bias: {param.data}")


Layer: encoder.0.0.block.0.weight
Weight: tensor([0.9750, 1.0008, 1.0394, 0.9690, 1.0381, 0.9551, 1.0361, 0.9804, 1.0251,
        0.9770, 0.9904, 1.0317, 1.0246, 0.9874, 1.0160, 0.9927, 1.0636, 0.9876,
        0.9794, 1.0233, 1.0472, 1.0000, 1.0550, 1.0249, 0.9639, 1.0036, 1.0061,
        1.0274, 1.0092, 1.0298, 1.0008, 1.0026, 1.0025, 1.0162, 0.9898, 1.0281,
        0.9993, 1.0172, 1.0028, 0.9791, 1.0002, 0.9492, 0.9808, 1.0047, 1.0121,
        1.0756, 1.0484, 1.0530, 1.0308, 1.0548, 1.0228, 0.9818, 1.0365, 1.0565,
        1.0072, 0.9981, 1.0169, 1.0323, 0.9796, 1.0250, 0.9712, 0.9878, 1.0699,
        1.0638, 1.0120, 0.9914, 0.9969, 1.0180, 0.9557, 0.9869, 0.9812, 1.0278,
        0.9913, 0.9831, 0.9790, 0.9776, 1.0698, 0.9919, 0.9821, 1.0128, 1.0208,
        0.9713, 1.0205, 0.9782, 1.0168, 0.9990, 1.0669, 0.9685, 0.9903, 1.0564,
        1.0028, 0.9732, 0.9900, 1.0016, 0.9764, 1.0538, 0.9700, 1.0321, 0.9853,
        1.0275, 0.9730, 1.0576, 1.0065, 0.9865, 0.9718, 1.0318, 0.9698, 1.0009

In [287]:
for var_name in model_quantized.state_dict():
    print(var_name, "\t", model_quantized.state_dict()[var_name])

positional_encoding 	 tensor([[ 0.7767,  0.4179, -0.9639,  ...,  1.0551,  0.6087,  0.1514],
        [-0.7247,  0.7949,  0.6012,  ..., -0.0981, -0.7312, -1.5987],
        [ 2.1631, -0.8226, -1.1655,  ..., -0.3369, -0.1976,  0.3002],
        ...,
        [ 0.2282,  0.4482, -0.0295,  ...,  0.1569, -0.5706, -0.3451],
        [ 0.2163, -0.3091, -1.6473,  ..., -1.4622,  0.0603,  0.7738],
        [-1.3462, -0.3745, -1.0837,  ...,  0.3060, -0.3337,  0.5932]])
encoder.0.0.block.0.weight 	 tensor([0.9750, 1.0008, 1.0394, 0.9690, 1.0381, 0.9551, 1.0361, 0.9804, 1.0251,
        0.9770, 0.9904, 1.0317, 1.0246, 0.9874, 1.0160, 0.9927, 1.0636, 0.9876,
        0.9794, 1.0233, 1.0472, 1.0000, 1.0550, 1.0249, 0.9639, 1.0036, 1.0061,
        1.0274, 1.0092, 1.0298, 1.0008, 1.0026, 1.0025, 1.0162, 0.9898, 1.0281,
        0.9993, 1.0172, 1.0028, 0.9791, 1.0002, 0.9492, 0.9808, 1.0047, 1.0121,
        1.0756, 1.0484, 1.0530, 1.0308, 1.0548, 1.0228, 0.9818, 1.0365, 1.0565,
        1.0072, 0.9981, 1.0169, 1.0

## Parameter Extraction

In [288]:
import numpy as np

model_state_dict = model_quantized.state_dict()

ignore_keys = [
    'scale',
    'zero_point',
    '_packed_params.dtype'
]

with open('model_layers_params.txt', 'w') as f:
    for layer_name, param_tensor in model_state_dict.items():
        if not any(ignore_key in layer_name for ignore_key in ignore_keys):
            if '_packed_params._packed_params' in layer_name:
                # Dynamically get the corresponding layer
                layer_parts = layer_name.split('.')
                layer = model_quantized
                for part in layer_parts[:-1]:
                    layer = getattr(layer, part)
                
                packed_params = getattr(layer, '_packed_params')
                
                # Unpack the quantized weights and biases
                int8_weights, int8_bias = torch.ops.quantized.linear_unpack(packed_params)
                int8_weights_nd = np.array(int8_weights.int_repr())
                int8_bias_nd = int8_bias.detach().numpy()
                
                f.write(f"Layer: {layer_name}\n")
                f.write(f"Quantized Weights:\n{int8_weights_nd}\n")
                f.write(f"Quantization Scale: {int8_weights.q_scale()}\n")
                f.write(f"Quantized Bias:\n{int8_bias_nd}\n")
                f.write("\n")
            else:
                # Convert param_tensor to ndarray
                param_ndarray = param_tensor.detach().numpy()
                
                f.write(f"Layer: {layer_name}\n") 
                f.write(f"Parameters:\n{param_ndarray}\n")
                f.write("\n")


測試提取參數是否正確

In [289]:
weight_tensor_after = model_quantized.state_dict()['encoder.2.1.block.1.2._packed_params._packed_params']

weight_tensor_after[0]

tensor([[-0.0196,  0.0141,  0.0685,  ...,  0.0087,  0.0435,  0.0120],
        [-0.0457, -0.0598,  0.0130,  ...,  0.0272,  0.0478, -0.0402],
        [ 0.0152, -0.0098, -0.0380,  ...,  0.0261,  0.0239,  0.0022],
        ...,
        [ 0.0174,  0.0152, -0.0076,  ..., -0.0033, -0.0544,  0.0163],
        [ 0.0163,  0.0380,  0.0315,  ..., -0.0272, -0.0370, -0.0370],
        [ 0.0239,  0.0043, -0.0109,  ...,  0.0098,  0.0228, -0.0359]],
       size=(192, 768), dtype=torch.qint8,
       quantization_scheme=torch.per_tensor_affine, scale=0.001087119453586638,
       zero_point=0)

### 提取六層 Encoder, Classifier, Embedding 參數

### 32-bit 浮點數

In [290]:
import os
import numpy as np
import torch

model_state_dict = model_quantized.state_dict()

ignore_keys = [
    '.scale',
    '.zero_point',
    '._packed_params.dtype'
]

def extract_packed_params(layer_name, param_tensor, folder_path):
    if '_packed_params._packed_params' in layer_name:
        layer_parts = layer_name.split('.')
        layer = model_quantized
        for part in layer_parts[:-1]:
            layer = getattr(layer, part)
        
        packed_params = getattr(layer, '_packed_params')
        
        int8_weights, int8_bias = torch.ops.quantized.linear_unpack(packed_params)
        int8_weights_nd = np.array(int8_weights.int_repr())
        int8_bias_nd = int8_bias.detach().numpy()
        
        # Save quantized weights
        weights_file = os.path.join(folder_path, 'quantized_weights.txt')
        with open(weights_file, 'w') as f:
            for weight in int8_weights_nd.flatten():
                f.write(f"{weight}\n")
        
        # Save quantized biases
        bias_file = os.path.join(folder_path, 'quantized_bias.txt')
        with open(bias_file, 'w') as f:
            np.savetxt(f, int8_bias_nd, fmt='%.8f')
        
        # Save quantization scale
        scale_file = os.path.join(folder_path, 'quantization_scale.txt')
        with open(scale_file, 'w') as f:
            f.write(str(int8_weights.q_scale()))
    else:
        param_ndarray = param_tensor.detach().numpy()
        
        # Save regular parameters
        param_file = os.path.join(folder_path, 'parameters.txt')
        with open(param_file, 'w') as f:
            np.savetxt(f, param_ndarray.flatten(), fmt='%.8f')

# Create the "32float" folder in the current directory
float32_folder = '32float'
os.makedirs(float32_folder, exist_ok=True)

# Extract encoder.0 to encoder.5
for i in range(6):
    folder_name = f'encoder_{i}_params'
    folder_path = os.path.join(float32_folder, folder_name)
    os.makedirs(folder_path, exist_ok=True)
    
    for layer_name, param_tensor in model_state_dict.items():
        if layer_name.startswith(f'encoder.{i}') and not any(layer_name.endswith(ignore_key) for ignore_key in ignore_keys):
            layer_folder = os.path.join(folder_path, layer_name.replace('.', '_'))
            os.makedirs(layer_folder, exist_ok=True)
            
            if '_packed_params._packed_params' in layer_name:
                extract_packed_params(layer_name, param_tensor, layer_folder)
            else:
                param_ndarray = param_tensor.detach().numpy()
                param_file = os.path.join(layer_folder, 'parameters.txt')
                with open(param_file, 'w') as f:
                    np.savetxt(f, param_ndarray.flatten(), fmt='%.8f')

# Extract remaining layers
layers_to_extract = ['classifier', 'embedding', 'positional_encoding']
for layer in layers_to_extract:
    folder_name = f'{layer}_params'
    folder_path = os.path.join(float32_folder, folder_name)
    os.makedirs(folder_path, exist_ok=True)
    
    for layer_name, param_tensor in model_state_dict.items():
        if layer_name.startswith(layer) and not any(layer_name.endswith(ignore_key) for ignore_key in ignore_keys):
            layer_folder = os.path.join(folder_path, layer_name.replace('.', '_'))
            os.makedirs(layer_folder, exist_ok=True)
            
            extract_packed_params(layer_name, param_tensor, layer_folder)


### 16-bit IEEE 754 binary format

- 整數轉成8位元的二進制
- 浮點數轉成16位元的IEEE 754二進制

In [291]:
import os
import numpy as np
import torch
import struct

model_state_dict = model_quantized.state_dict()

ignore_keys = [
    '.scale',
    '.zero_point',
    '._packed_params.dtype'
]

def float_to_bin_16bit(num):
    # Convert float to 16-bit IEEE 754 binary format
    binary = struct.pack('>e', num)
    return ''.join('{:08b}'.format(b) for b in binary)

def int_to_bin_8bit(num):
    # Convert integer to 8-bit signed binary format
    return '{:08b}'.format(num & 0xff)

def extract_packed_params(layer_name, param_tensor, folder_path):
    if '_packed_params._packed_params' in layer_name:
        layer_parts = layer_name.split('.')
        layer = model_quantized
        for part in layer_parts[:-1]:
            layer = getattr(layer, part)
        
        packed_params = getattr(layer, '_packed_params')
        
        int8_weights, int8_bias = torch.ops.quantized.linear_unpack(packed_params)
        int8_weights_nd = np.array(int8_weights.int_repr())
        int8_bias_nd = int8_bias.detach().numpy()
        
        # Save quantized weights as binary in text format
        weights_file = os.path.join(folder_path, 'quantized_weights.txt')
        with open(weights_file, 'w') as f:
            for weight in int8_weights_nd.flatten():
                f.write(int_to_bin_8bit(weight) + '\n')
        
        # Save quantized biases as binary in text format
        bias_file = os.path.join(folder_path, 'quantized_bias.txt')
        with open(bias_file, 'w') as f:
            for bias in int8_bias_nd.flatten():
                f.write(float_to_bin_16bit(bias) + '\n')
        
        # Save quantization scale as binary in text format
        scale_file = os.path.join(folder_path, 'quantization_scale.txt')
        with open(scale_file, 'w') as f:
            f.write(float_to_bin_16bit(int8_weights.q_scale()))
    else:
        param_ndarray = param_tensor.detach().numpy()
        
        # Save regular parameters as binary in text format
        param_file = os.path.join(folder_path, 'parameters.txt')
        with open(param_file, 'w') as f:
            for param in param_ndarray.flatten():
                if isinstance(param, np.integer):
                    f.write(int_to_bin_8bit(param) + '\n')
                else:
                    f.write(float_to_bin_16bit(param) + '\n')

# Extract encoder.0 to encoder.5
for i in range(6):
    folder_name = f'binary/encoder_{i}_params'
    os.makedirs(folder_name, exist_ok=True)
    
    for layer_name, param_tensor in model_state_dict.items():
        if layer_name.startswith(f'encoder.{i}') and not any(layer_name.endswith(ignore_key) for ignore_key in ignore_keys):
            layer_folder = os.path.join(folder_name, layer_name.replace('.', '_'))
            os.makedirs(layer_folder, exist_ok=True)
            
            if '_packed_params._packed_params' in layer_name:
                extract_packed_params(layer_name, param_tensor, layer_folder)
            else:
                param_ndarray = param_tensor.detach().numpy()
                param_file = os.path.join(layer_folder, 'parameters.txt')
                with open(param_file, 'w') as f:
                    for param in param_ndarray.flatten():
                        if isinstance(param, np.integer):
                            f.write(int_to_bin_8bit(param) + '\n')
                        else:
                            f.write(float_to_bin_16bit(param) + '\n')

# Extract remaining layers
layers_to_extract = ['classifier', 'embedding', 'positional_encoding']
for layer in layers_to_extract:
    folder_name = f'binary/{layer}_params'
    os.makedirs(folder_name, exist_ok=True)
    
    for layer_name, param_tensor in model_state_dict.items():
        if layer_name.startswith(layer) and not any(layer_name.endswith(ignore_key) for ignore_key in ignore_keys):
            layer_folder = os.path.join(folder_name, layer_name.replace('.', '_'))
            os.makedirs(layer_folder, exist_ok=True)
            
            extract_packed_params(layer_name, param_tensor, layer_folder)
